# Line Bot 必須要先申請帳號 才能得到secret跟token

# 可以參考這幾篇教學 就不贅述
# https://sites.google.com/jes.mlc.edu.tw/ljj/linebot%E5%AF%A6%E5%81%9A/%E5%A6%82%E4%BD%95%E7%94%B3%E8%AB%8Blinebot

# https://ithelp.ithome.com.tw/articles/10215268

# 接著我們先爬StockQ的資料 透過PANDAS 看表格

In [ ]:
有些人不確定你的資料類型 或是位置 可以先用print(type())或是  print(len())
這樣就知道你的表格位置比較快速找到 然後將資料存入到excel 就是這樣快速

In [ ]:
import pandas as pd
import requests 
import json 
import pandas as pd 
import numpy as np
def index():
    row_glod = pd.read_html("https://stockq.org/commodity/COMMGOLD.php")[9]
    row_glod.to_excel('glod.xlsx')

    #白銀
    row_silver = pd.read_html("https://stockq.org/commodity/COMMSILV.php")[9]
    row_silver.to_excel('silver.xlsx')

    #銅價
    row_copp = pd.read_html("https://stockq.org/commodity/COMMCOPP.php")[9]
    row_copp.to_excel('copp.xlsx')

    #天然氣
    row_gas = pd.read_html("https://stockq.org/commodity/FUTRNGAS.php")[9]
    row_gas.to_excel('gas.xlsx')

    #原油
    row_oil = pd.read_html("https://stockq.org/commodity/FUTRWOIL.php")[10]
    row_oil.to_excel('oil.xlsx')
    #黃豆
    row_soybean = pd.read_html("https://stockq.org/commodity/FUTRBEAN.php")[9]
    row_soybean.to_excel('soybean.xlsx')

    #那斯達克
    row_nasdaq= pd.read_html("https://stockq.org/index/NDX.php")[13]
    row_nasdaq.to_excel('nasdaq.xlsx')

    #道瓊
    row_dowj= pd.read_html("https://stockq.org/index/INDU.php")[18]
    row_dowj.to_excel('dowj.xlsx')

    #恐慌指數
    row_vix = pd.read_html("https://stockq.org/index/VIX.php")[15]
    row_vix.to_excel('vix.xlsx')

    #日經
    row_nkk = pd.read_html("https://stockq.org/index/NKY.php")[18]
    row_nkk.to_excel('nkk.xlsx')

    #越南
    row_vni= pd.read_html("https://stockq.org/index/VNINDEX.php")[17]
    row_vni.to_excel('vni.xlsx')

    #香港恆生
    row_hk = pd.read_html("https://stockq.org/index/HSI.php")[17]
    row_hk.to_excel('hk.xlsx')

    #印度指數
    row_india = pd.read_html("https://stockq.org/index/SENSEX.php")[18]
    row_india.to_excel('india.xlsx')

    #台灣加權
    row_twse = pd.read_html("https://stockq.org/index/TWSE.php")[17]
    row_twse.to_excel('twse.xlsx')



if __name__ == '__main__':
    index()

In [ ]:
這邊僅是複製前面天氣預報的爬蟲 就不需要視覺化只要把資料存入excel 會比存入sql快很快

In [ ]:
import requests,json,time
import plotly as py
from flask import Flask, render_template
import numpy as np
import pandas as pd

url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=CWB-BBDC4D80-BF90-4BE0-B721-D588CE9CD5E5'
data = requests.get(url)   # 取得 JSON 檔案的內容為文字
data_json = data.json()    # 轉換成 JSON 格式
location = data_json['records']['location']   # 取出 location 的內容
data=[]

for i in location:
    city = i['locationName']    # 縣市名稱
    mint8 = i['weatherElement'][2]['time'][0]['parameter']['parameterName']  # 最低溫
    maxt8 = i['weatherElement'][4]['time'][2]['parameter']['parameterName']  # 最高溫
    pop8 = i['weatherElement'][1]['time'][0]['parameter']['parameterName']   # 降雨機率  
    data.append([city,f'{mint8+"~"+maxt8}',pop8])
columns=['縣市','溫度預估','降雨機率 %']
df=pd.DataFrame(data,columns=columns)
df.to_excel('weather.xlsx')

In [ ]:
爬蟲都完成後我們就要開始設定LINE BOT
首先套入LINEBOT套件
可以

In [ ]:
pip install linebot

In [ ]:
from __future__ import unicode_literals
import os

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

import pandas as pd
import random,datetime,configparser
from flask import Flask, flash, redirect, render_template, request, abort

In [ ]:
這邊把你的token跟secret存入到同一個資料夾下取名config.ini裡面輸入
[line-bot]
channel_access_token=你的token
channel_secret=你的secret
當然也可以直接寫入看你怎麼設定


In [ ]:
app = Flask(__name__)

# LINE 聊天機器人的基本資料
config = configparser.ConfigParser()
config.read('config.ini')

line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token'))
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))

In [ ]:
完成後 就是要讓你可以接收到linebot的訊息
這邊就設定路徑是/callback 記得你的linebot api網址也要給 /callback
我之前也沒注意這個花了幾分鐘才發現

In [ ]:
# 接收 LINE 的資訊
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
這邊是當你接收到資料要去做處理
我們先做第一個股票部分 由於全世界股票不是天天開盤有時候時間不一樣 我們做一個字典
用datetime去看今天是星期幾 今天是不是有字典內的國家放假做提醒
這是為了避免有時候昨天大漲大跌 很多人以為今天要進場結果當日放假的停型

In [ ]:
msg=''
week_list = ["星期一","星期二","星期三","星期四","星期五","星期六","星期日"]
week=datetime.date.today()
week=week_list[week.weekday()]
today = datetime.date.today()
day = str(today).replace("-","/")
world={'美國':[(datetime.date(2022, 10, 10)),datetime.date(2022, 11, 11),datetime.date(2022, 11, 24),datetime.date(2022, 12, 26)],
       '台灣':[(datetime.date(2022, 10, 10))],
       '日本':[(datetime.date(2022, 10, 10)),datetime.date(2022, 11, 3),datetime.date(2022, 11, 23)],
       '香港':[(datetime.date(2022, 10, 4)),datetime.date(2022, 12, 26),datetime.date(2022, 12, 27)],
       '中國':[(datetime.date(2022, 10, 3)),datetime.date(2022, 10, 4),datetime.date(2022, 10, 5),datetime.date(2022, 10, 6),datetime.date(2022, 10, 7)],
       '新加坡':[(datetime.date(2022, 10, 24)),datetime.date(2022, 12, 26)],
       '印度':[(datetime.date(2022, 10, 5)),datetime.date(2022, 10, 24),datetime.date(2022, 11, 8)],
       '韓國':[(datetime.date(2022, 10, 3)),datetime.date(2022, 12, 30)]}

for festival,date in world.items():
    if today in date:
        msg+=f'今天 {festival} 休市不交易\n'
    else:
        msg+=''

In [ ]:
第二部分調出國際指數 原物料的資料 判斷昨日大漲大跌 我再原物料設定是 +/- 3%  指數則是+/-2%
有超過 低於才通知反之就沒有告知

In [ ]:
    glod=pd.read_excel('glod.xlsx')
    glods=glod.iloc[2][2]
    glod_date=glod.iloc[2][0].strip('2022/')
    if float(glods.strip('%'))> 3:
        msg+=f'{glod_date} 黃金 上漲幅度是:{glods}\n'
    elif float(glods.strip('%'))<-3:
        msg+=f'{glod_date} 黃金 下跌幅度是:{glods}\n'
    else:
        pass
    #白銀
    silver=pd.read_excel('silver.xlsx')
    silvers=silver.iloc[2][2]
    silver_date=silver.iloc[2][0].strip('2022/')
    if float(silvers.strip('%'))> 3:
        msg+=f'{silver_date} 白銀 上漲幅度是:{silvers}\n'
    elif float(silvers.strip('%'))<-3:
        msg+=f'{silver_date} 白銀 下跌幅度是:{silvers}\n'     
    else:
        pass

    #銅價
    copp=pd.read_excel('copp.xlsx')
    copps=copp.iloc[2][2]
    copp_date=copp.iloc[2][0].strip('2022/')
    if float(copps.strip('%'))> 3:
        msg+=f'{copp_date}銅價 上漲幅度是:{copps}\n'
    elif float(copps.strip('%'))<-3:
        msg+=f'{copp_date}銅價 下跌幅度是:{copps}\n'   
    else:
        pass

    #天然氣
    gas =pd.read_excel('gas.xlsx')
    gass=gas.iloc[2][2]
    gas_date=gas.iloc[2][0].strip('2022/')
    if float(gass.strip('%'))> 3:
        msg+=f'{gas_date} 天然氣 上漲幅度是:{gass}\n'
    elif float(gass.strip('%'))<-3:
        msg+=f'{gas_date} 天然氣 下跌幅度是:{gass}\n'    
    else:
        pass

    #原油
    oil=pd.read_excel('oil.xlsx')
    oils=oil.iloc[2][2]
    oil_date=oil.iloc[2][0].strip('2022/')
    if float(oils.strip('%'))> 3:
        msg+=f'{oil_date} 原油 上漲幅度是:{oils}\n'
    elif float(oils.strip('%'))<-3:
        msg+=f'{oil_date} 原油 下跌幅度是:{oils}\n'
    else:
        pass
    #黃豆
    soybean=pd.read_excel('soybean.xlsx')
    soybeans=soybean.iloc[2][2]
    soybean_date=soybean.iloc[2][0].strip('2022/')
    if float(soybeans.strip('%'))> 3:
        msg+=f'{soybean_date} 黃豆 上漲幅度是:{soybeans}\n'
    elif float(soybeans.strip('%'))<-3:
        msg+=f'{soybean_date} 黃豆 下跌幅度是:{soybeans}\n'  
    else:
        pass
    #恐慌指數
    row_vix = pd.read_excel('vix.xlsx')
    vix=row_vix.iloc[1][2]
    vix_date=row_vix.iloc[2][0].strip('2022/')
    if float(vix.strip('%'))> 3:
        msg+=f'{vix_date} Vix指數 上漲幅度是:{vix}\n'
    elif float(vix.strip('%'))<-3:
        msg+=f'{vix_date} Vix指數 下跌幅度是:{vix}\n'     
    else:
        pass
    msg+='\n'
    msg+='本通知商品包含:那斯達克、道瓊、台灣加權、香港恆生、越南、印度皆在台灣證交所有ETF，皆以正負 2% 幅度震盪通知僅供參考如有錯誤概不負法律責任\n'
    #nasdaq
    nasdaq=pd.read_excel('nasdaq.xlsx')
    nasdaqq=nasdaq.iloc[1][2]
    nasdaq_date=nasdaq.iloc[2][0].strip('2022/')
    nasdaqs=round(float(nasdaq.iloc[1][1])-float(nasdaq.iloc[2][1]))
    if float(nasdaqq.strip('%'))> 2:
        msg+=f'{nasdaq_date} 那斯達克 上漲幅度是:{nasdaqq},上漲點數 {nasdaqs} 點\n'
    elif float(nasdaqq.strip('%'))<-2:
        msg+=f'{nasdaq_date} 那斯達克 下跌幅度是:{nasdaqq},上漲點數 {nasdaqs} 點\n'    
    else:
        pass
    #道瓊
    dowj=pd.read_excel('dowj.xlsx')
    dowjj=dowj.iloc[1][2]
    dowj_date=dowj.iloc[2][0].strip('2022/')
    dowjjs=round(float(dowj.iloc[1][1])-float(dowj.iloc[2][1]))
    if float(dowjj.strip('%'))> 2:
        msg+=f'{dowj_date} 道瓊指數 上漲幅度是:{dowjj},上漲點數 {dowjjs} 點\n'
    elif float(dowjj.strip('%'))<-2:
        msg+=f'{dowj_date} 道瓊指數 下跌幅度是:{dowjj},下跌點數 {dowjjs} 點\n'    
    else:
        pass
    #日經
    nkk=pd.read_excel('nkk.xlsx')
    nkks=nkk.iloc[1][2]
    nkk_date=nkk.iloc[2][0].strip('2022/')
    nkk225=round(float(nkk.iloc[1][1])-float(nkk.iloc[2][1]))
    if float(nkks.strip('%'))> 2:
        msg+=f'{nkk_date} 日經指數 上漲幅度是:{nkks},上漲點數 {nkk225} 點\n'
    elif float(nkks.strip('%'))<-2:
        msg+=f'{nkk_date} 日經指數 下跌幅度是:{nkks},下跌點數 {nkk225} 點\n'     
    else:
        pass
    #越南
    row_vni=pd.read_excel('vni.xlsx')
    vni=row_vni.iloc[1][2]
    vni_date=row_vni.iloc[2][0].strip('2022/')
    vnii=round(float(row_vni.iloc[1][1])-float(row_vni.iloc[2][1]))
    if float(vni.strip('%'))> 2:
        msg+=f'{vni_date} 越南指數 上漲幅度是:{vni},上漲點數 {vnii} 點\n'
    elif float(vni.strip('%'))<-2:
        msg+=f'{vni_date} 越南指數 下跌幅度是:{vni},下跌點數 {vnii} 點\n'     
    else:
        pass
    #香港恆生
    row_hk=pd.read_excel('hk.xlsx')
    hk=row_hk.iloc[1][2]
    hk_date=row_hk.iloc[2][0].strip('2022/')
    hkk=round(float(row_hk.iloc[1][1])-float(row_hk.iloc[2][1]))
    if float(hk.strip('%'))> 2:
        msg+=f'{hk_date} 香港恆生 上漲幅度是:{hk},上漲點數 {hkk} 點\n'
    elif float(hk.strip('%'))<-2:
        msg+=f'{hk_date} 香港恆生 下跌幅度是:{hk},下跌點數 {hkk} 點\n'     
    else:
        pass
    #印度指數
    row_india=pd.read_excel('india.xlsx')
    india=row_india.iloc[1][2]
    india_date=row_india.iloc[2][0].strip('2022/')
    indias=round(float(row_india.iloc[1][1])-float(row_india.iloc[2][1]))
    if float(india.strip('%'))> 2:
        msg+=f'{india_date} 印度指數上漲幅度是:{india},上漲點數 {indias} 點\n'
    elif float(india.strip('%'))<-2:
        msg+=f'{india_date} 印度指數 下跌幅度是:{india},下跌點數 {indias} 點\n'   
    else:
        pass

    #台灣加權
    row_twse = pd.read_excel('twse.xlsx')
    twse=row_twse.iloc[1][2]
    twse_date=row_twse.iloc[2][0].strip('2022/')
    twses=round(float(row_twse.iloc[1][1])-float(row_twse.iloc[2][1]))
    if float(twse.strip('%'))> 2:
        msg+=f'{twse_date} 台灣加權指數上漲幅度是:{twse},上漲點數 {twses} 點\n'
    elif float(twse.strip('%'))<-2:
        msg+=f'{twse_date} 台灣加權指數 下跌幅度是:{twse},下跌點數 {twses} 點\n'    
    else:
        pass

In [ ]:
當然最後是接收的關鍵是 如果是股票則把前面的 是否開盤休市告知
如果接收到的是天氣  則告訴你天氣狀況 最後測試

想要改成固定時間去公布則可以用line notify這個設定好每天幾點固定通知

In [ ]:
#     以 requests 發送 POST 請求
if event.message.text =='股票':
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg))    
elif event.message.text =='天氣':
    row= pd.read_excel('weather.xlsx')
    msgs=''
    locA=f'{row.iloc[1][1]} 溫度預估:{row.iloc[1][2]} 度，降雨機率 {row.iloc[1][3]} %'
    msgs+=f'{locA}\n'
    locB=f'{row.iloc[13][1]} 溫度預估:{row.iloc[13][2]} 度，降雨機率 {row.iloc[13][3]} %'
    msgs+=f'{locB}\n'
    locC=f'{row.iloc[3][1]} 溫度預估:{row.iloc[3][2]} 度，降雨機率  {row.iloc[3][3]} %'
    msgs+=f'{locC}'
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msgs))

# Line Notify

In [ ]:
import requests
import datetime
import pandas as pd


def notify():
    # LINE Notify 權杖
    token = ''

    msg=''
    week_list = ["星期一","星期二","星期三","星期四","星期五","星期六","星期日"]
    week=datetime.date.today()
    week=week_list[week.weekday()]
    today = datetime.date.today()
    day = str(today).replace("-","/")
    world={'美國':[(datetime.date(2022, 10, 10)),datetime.date(2022, 11, 11),datetime.date(2022, 11, 24),datetime.date(2022, 12, 26)],
           '台灣':[(datetime.date(2022, 10, 10))],
           '日本':[(datetime.date(2022, 10, 10)),datetime.date(2022, 11, 3),datetime.date(2022, 11, 23)],
           '香港':[(datetime.date(2022, 10, 4)),datetime.date(2022, 12, 26),datetime.date(2022, 12, 27)],
           '中國':[(datetime.date(2022, 10, 3)),datetime.date(2022, 10, 4),datetime.date(2022, 10, 5),datetime.date(2022, 10, 6),datetime.date(2022, 10, 7)],
           '新加坡':[(datetime.date(2022, 10, 24)),datetime.date(2022, 12, 26)],
           '印度':[(datetime.date(2022, 10, 5)),datetime.date(2022, 10, 24),datetime.date(2022, 11, 8)],
           '韓國':[(datetime.date(2022, 10, 3)),datetime.date(2022, 12, 30)], }


    for festival,date in world.items():
        if today in date:
            msg+=f'今天 {festival} 休市不交易\n'
        else:
            msg+=''
    msg+='本通知商品包含:黃金、白銀、銅價、天然氣、原油、黃豆、Vix 皆在台灣證交所有ETF，以正負 3% 幅度震盪通知僅供參考如有錯誤概不負責。\n'
    #黃金
    glod=pd.read_excel('glod.xlsx')
    glod=glod.iloc[2][2]
    if float(glod.strip('%'))> 3:
        msg+=f'昨天黃金上漲幅度是:{glod}\n'
    elif float(glod.strip('%'))<-3:
        msg+=f'昨天黃金下跌幅度是:{glod}\n'
    else:
        pass
    #白銀
    silver=pd.read_excel('silver.xlsx')
    silver=silver.iloc[2][2]
    if float(silver.strip('%'))> 3:
        msg+=f'昨天白銀上漲幅度是:{silver}\n'
    elif float(silver.strip('%'))<-3:
        msg+=f'昨天白銀下跌幅度是:{silver}\n'     
    else:
        pass

    #銅價
    copp=pd.read_excel('copp.xlsx')
    copp=copp.iloc[2][2]
    if float(copp.strip('%'))> 3:
        msg+=f'昨天銅價上漲幅度是:{copp}\n'
    elif float(silver.strip('%'))<-3:
        msg+=f'昨天銅價下跌幅度是:{copp}\n'   
    else:
        pass

    #天然氣
    gas =pd.read_excel('gas.xlsx')
    gas=gas.iloc[2][2]
    if float(gas.strip('%'))> 3:
        msg+=f'昨天天然氣上漲幅度是:{gas}\n'
    elif float(gas.strip('%'))<-3:
        msg+=f'昨天天然氣下跌幅度是:{gas}\n'    
    else:
        pass

    #原油
    oil=pd.read_excel('oil.xlsx')
    oil=oil.iloc[2][2]
    if float(oil.strip('%'))> 3:
        msg+=f'昨天原油上漲幅度是:{oil}\n'
    elif float(gas.strip('%'))<-3:
        msg+=f'昨天原油下跌幅度是:{oil}\n'
    else:
        pass
    #黃豆
    soybean=pd.read_excel('soybean.xlsx')
    soybean=soybean.iloc[2][2]
    if float(soybean.strip('%'))> 3:
        msg+=f'昨天黃豆上漲幅度是:{soybean}\n'
    elif float(soybean.strip('%'))<-3:
        msg+=f'昨天黃豆下跌幅度是:{soybean}\n'  
    else:
        pass
    #恐慌指數
    row_vix = pd.read_excel('vix.xlsx')
    vix=row_vix.iloc[1][2]
    if float(vix.strip('%'))> 3:
        msg+=f'昨天 Vix指數 上漲幅度是:{vix}\n'
    elif float(vix.strip('%'))<-3:
        msg+=f'昨天 Vix指數 下跌幅度是:{vix}\n'     
    else:
        pass
    msg+='本通知商品包含:那斯達克、道瓊、台灣加權、香港恆生、越南、印度皆在台灣證交所有ETF，皆以正負 2% 幅度震盪通知僅供參考如有錯誤概不負法律責任\n'
    #nasdaq
    nasdaq=pd.read_excel('nasdaq.xlsx')
    nasdaqq=nasdaq.iloc[1][2]
    nasdaqs=round(float(nasdaq.iloc[1][1])-float(nasdaq.iloc[2][1]))
    if float(nasdaqq.strip('%'))> 2:
        msg+=f'昨天那斯達克上漲幅度是:{nasdaqq},上漲點數 {nasdaqs} 點\n'
    elif float(nasdaqq.strip('%'))<-2:
        msg+=f'昨天那斯達克下跌幅度是:{nasdaqq},上漲點數 {nasdaqs} 點\n'    
    else:
        pass
    #道瓊
    dowj=pd.read_excel('dowj.xlsx')
    dowjj=dowj.iloc[1][2]
    dowjjs=round(float(dowj.iloc[1][1])-float(dowj.iloc[2][1]))
    if float(dowjj.strip('%'))> 2:
        msg+=f'昨天道瓊指數上漲幅度是:{dowjj},上漲點數 {dowjjs} 點\n'
    elif float(dowjj.strip('%'))<-2:
        msg+=f'昨天道瓊指數下跌幅度是:{dowjj},下跌點數 {dowjjs} 點\n'    
    else:
        pass
    #日經
    nkk=pd.read_excel('nkk.xlsx')
    nkks=nkk.iloc[1][2]
    nkk225=round(float(nkk.iloc[1][1])-float(nkk.iloc[2][1]))
    if float(nkks.strip('%'))> 2:
        msg+=f'昨天 日經指數 上漲幅度是:{nkks},上漲點數 {nkk225} 點\n'
    elif float(nkks.strip('%'))<-2:
        msg+=f'昨天 日經指數 下跌幅度是:{nkks},下跌點數 {nkk225} 點\n'     
    else:
        pass
    #越南
    row_vni=pd.read_excel('vni.xlsx')
    vni=row_vni.iloc[1][2]
    vnii=round(float(row_vni.iloc[1][1])-float(row_vni.iloc[2][1]))
    if float(vni.strip('%'))> 2:
        msg+=f'昨天 越南指數 上漲幅度是:{vni},上漲點數 {vnii} 點\n'
    elif float(vni.strip('%'))<-2:
        msg+=f'昨天 越南指數 下跌幅度是:{vni},下跌點數 {vnii} 點\n'     
    else:
        pass
    #香港恆生
    row_hk=pd.read_excel('hk.xlsx')
    hk=row_hk.iloc[1][2]
    hkk=round(float(row_hk.iloc[1][1])-float(row_hk.iloc[2][1]))
    if float(hk.strip('%'))> 2:
        msg+=f'昨天 香港恆生 上漲幅度是:{hk},上漲點數 {hkk} 點\n'
    elif float(hk.strip('%'))<-2:
        msg+=f'昨天 香港恆生 下跌幅度是:{hk},下跌點數 {hkk} 點\n'     
    else:
        pass
    #印度指數
    row_india=pd.read_excel('india.xlsx')
    india=row_india.iloc[1][2]
    indias=round(float(row_india.iloc[1][1])-float(row_india.iloc[2][1]))
    if float(india.strip('%'))> 2:
        msg+=f'昨天 印度指數上漲幅度是:{india},上漲點數 {indias} 點\n'
    elif float(india.strip('%'))<-2:
        msg+=f'昨天 印度指數 下跌幅度是:{india},下跌點數 {indias} 點\n'   
    else:
        pass

    #台灣加權
    row_twse = pd.read_excel('twse.xlsx')
    twse=row_twse.iloc[1][2]
    twses=round(float(row_twse.iloc[1][1])-float(row_twse.iloc[2][1]))
    if float(twse.strip('%'))> 2:
        msg+=f'昨天 台灣加權指數上漲幅度是:{twse},上漲點數 {twses} 點\n'
    elif float(twse.strip('%'))<-2:
        msg+=f'昨天 台灣加權指數 下跌幅度是:{twse},下跌點數 {twses} 點\n'    
    else:
        pass
    # HTTP 標頭參數與資料
    headers = { "Authorization": "Bearer " + token }
    data = { 'message': msg }

    # 以 requests 發送 POST 請求
    requests.post("https://notify-api.line.me/api/notify",headers = headers, data = data)
if __name__ == "__main__":
    notify()